# Build model

In [1]:
import torch
from torchtext.datasets import AG_NEWS
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import Vocab
from torch.utils.data import DataLoader
from torch import nn
import time
from torch.utils.data.dataset import random_split

In [2]:
train_iter = AG_NEWS(split='train')

In [3]:
tokenizer = get_tokenizer('basic_english')
train_iter = AG_NEWS(split='train')
counter = Counter()
for (label, line) in train_iter:
    counter.update(tokenizer(line))
vocab = Vocab(counter, min_freq=1)

In [4]:
text_pipeline = lambda x: [vocab[token] for token in tokenizer(x)]
label_pipeline = lambda x: int(x)

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

train_iter = AG_NEWS(split='train')
dataloader = DataLoader(train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch)

In [6]:
class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)


In [7]:
train_iter = AG_NEWS(split='train')
num_class = len(set([label for (label, text) in train_iter]))
vocab_size = len(vocab)
emsize = 64
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

In [8]:
def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predited_label = model(text, offsets)
        loss = criterion(predited_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predited_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predited_label = model(text, offsets)
            loss = criterion(predited_label, label)
            total_acc += (predited_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

In [9]:
from torch.utils.data.dataset import random_split
# Hyperparameters
EPOCHS = 2 # epoch
LR = 5  # learning rate
BATCH_SIZE = 64 # batch size for training

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None
train_iter, test_iter = AG_NEWS()
train_dataset = list(train_iter)
test_dataset = list(test_iter)
# Make labels zero indexed
test_dataset = [(a - 1, b) for a, b in test_dataset]
train_dataset = [(a - 1, b) for a, b in train_dataset]

num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = \
    random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
        scheduler.step()
    else:
        total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

| epoch   1 |   500/ 1782 batches | accuracy    0.682
| epoch   1 |  1000/ 1782 batches | accuracy    0.854
| epoch   1 |  1500/ 1782 batches | accuracy    0.875
-----------------------------------------------------------
| end of epoch   1 | time: 10.36s | valid accuracy    0.880 
-----------------------------------------------------------
| epoch   2 |   500/ 1782 batches | accuracy    0.897
| epoch   2 |  1000/ 1782 batches | accuracy    0.899
| epoch   2 |  1500/ 1782 batches | accuracy    0.903
-----------------------------------------------------------
| end of epoch   2 | time: 10.19s | valid accuracy    0.892 
-----------------------------------------------------------


In [10]:
print('Checking the results of test dataset.')
accu_test = evaluate(test_dataloader)
print('test accuracy {:8.3f}'.format(accu_test))

Checking the results of test dataset.
test accuracy    0.898


# PREDICT FUNCTION

In [11]:
sm = torch.nn.Softmax()
ag_news_label = {1: "World",
                 2: "Sports",
                 3: "Business",
                 4: "Sci/Tec"}
ex_text_str = "MEMPHIS, Tenn. – Four days ago, Jon Rahm was \
    enduring the season’s worst weather conditions on Sunday at The \
    Open on his way to a closing 75 at Royal Portrush, which \
    considering the wind and the rain was a respectable showing. \
    Thursday’s first round at the WGC-FedEx St. Jude Invitational \
    was another story. With temperatures in the mid-80s and hardly any \
    wind, the Spaniard was 13 strokes better in a flawless round. \
    Thanks to his best putting performance on the PGA Tour, Rahm \
    finished with an 8-under 62 for a three-stroke lead, which \
    was even more impressive considering he’d never played the \
    front nine at TPC Southwind."

In [14]:
def predict_fn(model, texts, tokenizer, vocab):
    with torch.no_grad():
        texts = [
            torch.tensor(
                [vocab[token] for token in tokenizer(text)]
            ) 
            for text in texts]
        text_list = torch.tensor(torch.cat(texts)).long()
        
        offsets = [0]
        for text in texts:
            offsets.append(text.size(0))
        offsets = torch.tensor(offsets[:-1]).cumsum(dim=0).long()
        
        output = model(text_list, offsets)
        
        return sm(output).numpy().tolist()

In [15]:
predict_fn(model, [ex_text_str, "two"], tokenizer, vocab)

/var/folders/_6/xxzr8rcn1pd4769rhd8244n00000gn/T/ipykernel_9141/735817920.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  text_list = torch.tensor(torch.cat(texts)).long()
/var/folders/_6/xxzr8rcn1pd4769rhd8244n00000gn/T/ipykernel_9141/735817920.py:17: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return sm(output).numpy().tolist()


[[0.02745627425611019,
  0.9555160403251648,
  0.005490866955369711,
  0.01153685711324215],
 [0.16607433557510376,
  0.2975232005119324,
  0.002381894737482071,
  0.5340205430984497]]

# UNBOX

In [16]:
import unboxapi
# client = unboxapi.UnboxClient("YOUR_API_KEY_HERE")
unboxapi.DEPLOYMENT = unboxapi.DeploymentType.AWS
unboxapi.api.UNBOX_ENDPOINT = "http://0.0.0.0:8080/api" # Use this for local testing
client = unboxapi.UnboxClient("481bb19f-c58c-415b-a957-36cbfe17ec85")

# Package & Upload to Unbox

### Upload model

In [17]:
from unboxapi.models import ModelType

model = client.add_model(
    function=predict_fn, 
    model=model,
    model_type=ModelType.pytorch,
    class_names=['world', 'sports', 'business', 'sci/tec'],
    name='pytorch 4',
    description='this is my pytorch model',
    requirements_txt_file='./requirements.txt',
    tokenizer=tokenizer,
    vocab=vocab,
)
model.to_dict()

[2021-07-07 17:17:48,528] WARNING - BentoML by default does not include spacy and torchvision package when using PytorchModelArtifact. To make sure BentoML bundle those packages if they are required for your model, either import those packages in BentoService definition file or manually add them via `@env(pip_packages=['torchvision'])` when defining a BentoService


  0%|                                                                                                                                                                           | 0.00/22.7M [00:00<?, ?B/s]

Connecting to Unbox server


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 22.7M/22.7M [00:14<00:00, 1.60MB/s]


{'_links': {'datasets': '/api/models/4e0c3df3-5a22-4934-942b-8b78798db898/datasets',
  'inferenceRuns': '/api/models/4e0c3df3-5a22-4934-942b-8b78798db898/inference-runs',
  'runReports': '/api/models/4e0c3df3-5a22-4934-942b-8b78798db898/run-reports',
  'self': '/api/models/4e0c3df3-5a22-4934-942b-8b78798db898'},
 'classNames': ['world', 'sports', 'business', 'sci/tec'],
 'datasetCount': 0,
 'dateCreated': '2021-07-08T00:18:06.797474Z',
 'description': 'this is my pytorch model',
 'id': '4e0c3df3-5a22-4934-942b-8b78798db898',
 'inProgressNotifications': ['model_4e0c3df3-5a22-4934-942b-8b78798db898_create_endpoint'],
 'inferenceRunCount': 0,
 'modelApi': None,
 'name': 'pytorch 4',
 'runReportCount': 0}

### Upload dataset from dataframe

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(test_dataset, columns=["label", "text"])[:1000]

In [ ]:
dataset = client.add_dataframe(
    df=df,
    class_names=['world', 'sports', 'business', 'sci/tec'],
    label_column_name='label',
    text_column_name='text',
    name='Pytorch test dataset',
    description='this is my Pytorch test dataset'
)
dataset.to_dict()